<a href="https://colab.research.google.com/github/sravika-reddy12/Sravika-Reddy_INFO5731_Spring2023/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **INFO 5731 ASSIGNMENT 2**

1 Write a python program to collect text data from either of the following sources and save the data into a csv file:(40 points)

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter.

[ ]


In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

# specify the URL of the IMDB page for the Avatar movie reviews
url = 'https://www.imdb.com/title/tt1630029/reviews/?ref_=tt_ql_urv'

# send a GET request to the URL
response = requests.get(url) # parse the HTML content using Beautiful Soup


soup = BeautifulSoup(response.content, 'html.parser') # find the total number of user reviews


total_reviews_str = soup.find('div', {'class': 'header'}).get_text()
total_reviews_str = total_reviews_str.split()[0] # extract the first word of the string

total_reviews = int(total_reviews_str.replace(',', ''))


csv_file = open('Avatar_reviews.csv', 'w', newline='', encoding='utf-8') # create a CSV file to save the data
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Review'])

# loop through all the pages of reviews and save them in the CSV file
for page in range(1, min(1001, total_reviews // 10 + 2)):# generate the URL for the current page of reviews
    
    page_url = f'{url}&start={10*(page-1)}'
    
    page_response = requests.get(page_url)# send a GET request to the page URL
   
    page_soup = BeautifulSoup(page_response.content, 'html.parser') # parse the HTML content using Beautiful Soup
    
    reviews = page_soup.find_all('div', {'class': 'text show-more__control'}) # find all the user reviews on the page
    
    for review in reviews: # loop through all the reviews and save them in the CSV file
        review_text = review.get_text().strip()
        csv_writer.writerow([review_text])

# close the CSV file
csv_file.close()



In [ ]:
import pandas as pd

                   # read the CSV file into a DataFrame
df = pd.read_csv('Avatar_reviews.csv')

                   # print the first 100 rows of the DataFrame
print(df.head(3000))

2. Write a python program to clean the text data you collected above and save the data in a new column in the csv file. The data cleaning steps include:(30 points).

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
# Write your code here
import nltk
nltk.download()


In [ ]:
#Remove punctuations 
#and special characters

import re # importing
#PUNCTUATIONS
df['Reviews after Noise Removal'] = df['Review'].str.replace('[^\w\s]','')
df['Reviews after Noise Removal'] = df['Reviews after Noise Removal'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

In [ ]:
#Removing -digits
#DIGITS
df['After digits removal'] = df['Reviews after Noise Removal'].apply(lambda y: ''.join([i for i in y if not i.isdigit()]))

In [ ]:
#Removing -stop words

from nltk.corpus import stopwords
#STOP WORDS
s = stopwords.words('english')
df['Stopwords Removal'] = df['After digits removal'].apply(lambda x: " ".join(x for x in x.split() if x not in s))

In [ ]:
#Convert to lower case
#LOWER CASE
df['Lower Case'] = df['Stopwords Removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df

In [ ]:
#Stemming

from nltk.stem import PorterStemmer
s = PorterStemmer()  
#STEMMING
df['After Stemming'] = df['Lower Case'].apply(lambda x: " ".join([s.stem(word) for word in x]))

In [ ]:
#Lemmatization

from textblob import Word
import nltk   #importing
nltk.download('wordnet')
#LEMMATIZATION

df['After Lemmatization'] = df['After Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [ ]:
df.to_csv('Avatar1_reviews.csv', index=False)

3.Write a python program to conduct syntax and structure analysis of the clean text you just saved above. The syntax and structure analysis includes:(30 points). 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# Write your code here

from nltk.tokenize import word_tokenize
p = []
#for statement
for sentence in df['Lower Case']:
  text = word_tokenize(sentence)
  p.append(nltk.pos_tag(text))
p



**3.2Constituency** **Parsing**

In [ ]:
import tensorflow as tf

In [ ]:
%tensorflow_version 2.x
#advanced version
import tensorflow as tf
#tensorflow
import benepar   
# benepar
benepar.download('benepar_en3')

In [ ]:
import spacy.cli
#spacy.cli
spacy.cli.download("en_core_web_sm")

In [ ]:
import benepar
parser = benepar.Parser("benepar_en3")
for sentence in df['Lower Case']:
  try:
    tree = parser.parse(sentence)
    print(tree)
  except:
    print("No Parse Tree")
    continue

In [ ]:
#Depedency Parsing

import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
options = {'compact': True, 'font': 'Source Sans Pro', 'distance': 100}
for sentence in df['Lower Case']:
  doc = nlp(sentence)
  displacy.render(doc, style = 'dep', options=options, jupyter=True)

**3.3 Named Entity Recognition**

In [ ]:
#Named Entity Recognition

import en_core_web_sm
nlp = en_core_web_sm.load()

for sentence in df['Lower Case']:
  doc = nlp(sentence)
  for X in doc.ents:
    if X.text and X.label_:
      print([(X.text, X.label_)])

Here (Question 3-2), please explain the constituency parsing tree and dependency parsing tree.

In [ ]:
'''Constituent parsing, also known as syntactical parsing, is the process of breaking down a text into smaller sub-texts and sub-phrases.
This kind of parsing identifies the syntactical organization of the sentence under consideration. Sentence components, parts of speech tags, and sentence words make up this list.
As an illustration, consider (S (NP (PRP I (ADVP (RB never))(VP (VBD left) (NP

(SBAR (S (NP (PRP I (VP (VP (VBD registered) (NNP imdb) (NN account)) (NP (NN review) (NN title)) (NP (JJ good) (JJ bad))
(Vent NNP) (NN opinion) (SBAR (S (NP (PRP I (VP (VBP want) (NP (NNS hours) (NN life)) (X (RB back)))))))))))))))))))))
By treating each word as a node, dependency parsing describes the grammatical structure of a chosen phrase.
Dependency grammar is used in this kind of parsing. Each word in this sentence is connected using their relationships.
Intelligence, skills, metally, and illness are all present in the dependency parsing tree shown above. They both depend on talent, which comes before intelligence.'''